## Introduction

This module is developed not only to annotate the shapes of tensors of DL models, <br/>
but also to test the annotated shapes.

Therefore, the default test algorithms expects the inputs with `.shape: Set[int]` field.
I expect, we can customize 3 functions. They are:

- get_meta_annotations
- generate_meta_dict
- push_execution_result

You might feel the implementations of the first two are kind of advanced.
Since I do the `Microwise Development`, it would be easy to track the mechanism.

To use the `test_meta_dict`, we need to prepare two things.

- comparison expressions
- arguments in the expressions

You would modify the functions to manipulate the algorithm to get those values.


## Code Example


### Understanding of Types and Functions

First, look through to `MetaDict_`.

In [1]:
from crimson.executable_types_beta.utils import (
    get_meta_annotations,
    generate_meta_dict,
    delegate_func_returning_args_details
)
from crimson.types_beta.addon import TypesPack
from crimson.executable_types_beta.mock import TensorMock
from crimson.executable_types_beta.executor import AnnotationExecutor
from crimson.executable_types_beta.test import test_meta_dict
from typing import List
from crimson.executable_types_beta.types import *
from inspect import currentframe

In [2]:
array1 = TensorMock.ones((4, 6))
array2 = TensorMock.ones((8, 10))
array3 = TensorMock.ones((4, 8))

In [3]:
def func(
    array1: TensorMock[List, "b", "c"], 
    array2: TensorMock[List, "d == 2*b", "e == b+2*c"],
    array3: TensorMock
) -> TensorMock[List, "b", "f == 3 * b"]:
    array4 = TensorMock.ones((4, 8))
    return array4

get_meta_annotations(func=func)

{'array1': ['b', 'c'],
 'array2': ['d == 2*b', 'e == b+2*c'],
 'array3': None,
 'return': ['b', 'f == 3 * b']}

In [4]:
output, arg_details, model_fields = delegate_func_returning_args_details(
    func=func,
    env=currentframe().f_locals,
	**{
        'array1':array1,
        'array2':array2,
        'array3':array3,
    }
)

In [5]:
output

[[1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]

In [6]:
arg_details: ArgsDetails_ = arg_details
arg_details

{'array1': [[1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1]],
 'array2': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'array3': [[1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]],
 'return': [[1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]]}

In [7]:
meta_annotations: MetaAnnotations_ = get_meta_annotations(func)
meta_annotations

{'array1': ['b', 'c'],
 'array2': ['d == 2*b', 'e == b+2*c'],
 'array3': None,
 'return': ['b', 'f == 3 * b']}

In [8]:
meta_dict: MetaDict_ = generate_meta_dict(
    args_details=arg_details,
    meta_annotations=meta_annotations
)

meta_dict

{'b': 4, 'c': 6, 'd': 8, 'e': 10, 'f': 8}

### Test

As default, <br/>
We get those `comparison expression`s as `MetaAnnotations_`. <br/>
To test those expressions, we need the arguments. <br/>
The arguments are provided as `MetaDict_`

Finally, `test_meta_dict` function tests the expressions.

In [9]:
test_result:TestResult_ = test_meta_dict(
    meta_dict=meta_dict,
    meta_annotations=meta_annotations
)

test_result

{'d == 2 * b': True, 'e == b + 2 * c': False, 'f == 3 * b': False}